In [1]:
from model import GPT
import json
from safetensors.torch import load_model
from inference import generate, text_to_token_ids, token_ids_to_text
import sentencepiece as spm

In [ ]:
directory = ""

In [6]:
with open(f"{directory}config.json") as f:
    config = json.load(f)["model"]

model = GPT(config)

load_model(model, f"{directory}model.safetensors")

(set(), [])

In [7]:
tokenizer = spm.SentencePieceProcessor(
    model_file=f'tokenizers/tinynewstokenizer{config["vocab_size"]}.model')

### Special Tokens

- [0] : Unknown
- [1] : BOS - This is prepended by `text_to_token_ids(...)`
- [2] : EOS - This is used by `generate(...)` to stop inference
- [3] : `</h>` - This token marks the end of the headline

### Generating its own headlines:

In [18]:
token_ids = generate(
    model=model,
    idx=text_to_token_ids("", tokenizer),
    context_length=config["context_length"],
    top_k=32,
    temperature=1,
    eos_id=2
)

print(token_ids_to_text(token_ids, tokenizer).replace("</h>", "\n" + "-" * len(headline) + "\n").replace(".", "\n"))

government launches new policies to boost economy 
------------------------------
 the government has launched new policies aimed at boosting the economy
 the policies are designed to reduce the country's debt and create more jobs
 the policies include cutting back on unnecessary jobs and increasing government services
 many are skeptical about the policies, but many are optimistic about the potential for positive changes
 the policies are expected to come into effect in the coming months



### Headline as a prompt:

In [13]:
headline = "local computer learns to think"

token_ids = generate(
    model=model,
    idx=text_to_token_ids(headline.lower() + " </h>", tokenizer),
    context_length=config["context_length"],
    top_k=None,
    temperature=0,
    eos_id=2
)

print(token_ids_to_text(token_ids, tokenizer).replace("</h>", "\n" + "-" * len(headline) + "\n").replace(".", "\n"))

local computer learns to think 
------------------------------
 a local computer program has been developed to help people learn to think
 the program uses interactive games and activities to teach people how to solve problems and stay motivated
 the program has been a huge success, with many people reporting that it has helped them to improve their skills and knowledge
 the program is expected to continue to grow and evolve

